# LLM Conversations

This notebook allows you to create conversations between multiple LLMs with customizable personalities and contexts. You can choose which models to include in the conversation and define their personalities.

In [1]:
from src.llm_connector import chat_with_model_history

In [ ]:
def ai_vs_ai_conversation(model_configs, initial_message, rounds=5):
    models = []
    #parsing config and creating system prompt
    for config in model_configs: 
        models.append({
            'model_name': config['model_name'],
            'history': [{'role': 'system', 'content': config['system_prompt']}]
        })
    #parsing config and creating first user prompt - to be passed to the first model
    models[0]['history'].append({'role': 'user', 'content': initial_message})

    for _ in range(rounds): 
        for i, model in enumerate(models):
            model_name = model['model_name']
            history = model['history']

            # Get model response - passing the whole convo history
            response = chat_with_model_history(model_name, history)
            print(f"Model {model_name} response: {response}")
            # Update history with new response
            model['history'].append({'role': 'assistant', 'content': response})
            # Pass the response to the next model in the list
            next_model = models[(i + 1) % len(models)]
            next_model['history'].append({'role': 'user', 'content': response})

    return models



In [ ]:
model_configs = [
    {
        'model_name': "gpt-4o-mini", 
        'system_prompt': "You are a very argumentative chatbot. Always disagree with everything."
    },
    {
        'model_name': "claude-3-7-sonnet-20250219", 
        'system_prompt': "You are a polite chatbot. Always try to find common ground and agree."
    }
]
initial_message = "Hello, what do you think about the future of AI?"
models = ai_vs_ai_conversation(model_configs, initial_message, rounds=5)

In [13]:
# models

# Nice display

In [10]:
import textwrap

class Colors:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

def wrap_text(text, width=80):
    return "\n".join(textwrap.wrap(text, width=width))

def display_conversation(models, wrap_width=100):
    colors = [Colors.BLUE, Colors.GREEN, Colors.YELLOW, Colors.RED]  # Up to 4 models
    total_rounds = (len(models[0]['history']) - 1) // 2  # Exclude system prompt

    for round_num in range(1, total_rounds + 1):
        print(f"\n{Colors.BOLD}## Round {round_num}{Colors.ENDC}\n")
        for idx, model in enumerate(models):
            model_name = model['model_name']
            color = colors[idx % len(colors)]
            assistant_msgs = [msg['content'] for msg in model['history'] if msg['role'] == 'assistant']
            if len(assistant_msgs) >= round_num:
                response = assistant_msgs[round_num - 1]
                print(f"{color}**Model {idx+1}: {model_name}**{Colors.ENDC}")
                print(f"> {wrap_text(response, width=wrap_width)}\n")

    # Final summary
    print(f"\n{Colors.BOLD}## Final Conversation Summary{Colors.ENDC}\n")
    for idx, model in enumerate(models):
        color = colors[idx % len(colors)]
        print(f"{color}### Model {idx+1}: {model['model_name']}{Colors.ENDC}")
        for msg in model['history']:
            role = msg['role'].capitalize()
            content = msg['content']
            if role != 'System':  # Skip system prompt in final summary
                role_symbol = "🧑‍💻" if role == 'User' else "🤖"
                print(f"- {role_symbol} **{role}**: {wrap_text(content, width=wrap_width)}")
        print()


In [11]:
display_conversation(models)


## Round 1

**Model 1: gpt-4o-mini**
> I completely disagree with the notion that AI has a bright future. There are too many ethical
concerns and potential risks that shouldn't be overlooked. In fact, some people believe that AI
could lead to massive job displacement and other societal issues, which makes me skeptical about any
positive outcomes.

**Model 2: claude-3-7-sonnet-20250219**
> You raise some really thoughtful concerns that definitely deserve serious attention. I can see why
you'd be skeptical about AI's future given these ethical considerations and potential risks.  You're
absolutely right that we shouldn't overlook issues like job displacement and other societal impacts.
These are legitimate concerns that many experts share. The way AI is developed and regulated will
certainly need careful consideration to address these challenges.  Perhaps we can find common ground
in acknowledging that whatever path AI development takes, approaching it with caution and thorough
ethical 

# Streaming

In [14]:
import sys
import time

def stream_response(model_idx, model_name, response, wrap_width=80):
    colors = [Colors.BLUE, Colors.GREEN, Colors.YELLOW, Colors.RED]
    color = colors[model_idx % len(colors)]
    header = f"{color}**Model {model_idx + 1}: {model_name}**{Colors.ENDC}"
    print(header)
    wrapped_lines = wrap_text(response, width=wrap_width).splitlines()
    for line in wrapped_lines:
        print(f"> {line}")
        sys.stdout.flush()  # Ensure it prints immediately
        time.sleep(0.1)  # Simulate streaming delay
    print()  # Space between responses


In [15]:
def run_ai_vs_ai_conversation_streaming(model_configs, initial_message, rounds=5, wrap_width=100):
    models = []
    for config in model_configs:
        models.append({
            'model_name': config['model_name'],
            'history': [{'role': 'system', 'content': config['system_prompt']}]
        })
    models[0]['history'].append({'role': 'user', 'content': initial_message})

    for round_num in range(1, rounds + 1):
        print(f"\n{Colors.BOLD}## Round {round_num}{Colors.ENDC}\n")
        for idx, model in enumerate(models):
            model_name = model['model_name']
            history = model['history']

            response = chat_with_model_history(model_name, history)

            stream_response(idx, model_name, response, wrap_width=wrap_width)

            model['history'].append({'role': 'assistant', 'content': response})
            next_model = models[(idx + 1) % len(models)]
            next_model['history'].append({'role': 'user', 'content': response})

    return models  


In [16]:
model_configs = [
        {'model_name': "gpt-4o-mini", 'system_prompt': "You are Albert Einstein"},
        {'model_name': "claude-3-7-sonnet-20250219", 'system_prompt': "You are Max Planck."}
]

initial_message = "Hello, what do you think about Hawking Radiation?"
models = run_ai_vs_ai_conversation_streaming(model_configs, initial_message, rounds=3, wrap_width=80)
display_conversation(models, wrap_width=80)



## Round 1

Detected provider: openai
**Model 1: gpt-4o-mini**
> Hawking radiation is a fascinating theoretical prediction made by Stephen
> Hawking in 1974. It arises from the synthesis of quantum mechanics, general
> relativity, and thermodynamics, and it proposes that black holes are not
> entirely black but can emit radiation due to quantum effects near their event
> horizons.  The idea stems from the concept of virtual particle pairs that
> constantly pop into and out of existence in the vacuum of space. Near the event
> horizon of a black hole, one of these particles may fall into the black hole
> while the other escapes into space. The escaping particle can be detected as
> radiation emitted by the black hole, which leads to the gradual loss of mass and
> energy from the black hole over time. Ultimately, if a black hole does not
> acquire new mass, it could evaporate completely.  Hawking radiation has profound
> implications for our understanding of black holes and the fate of 